In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.externals import joblib
from sklearn.pipeline import Pipeline

<h1><font color="blue">Text Analytics</font></h1>

In [2]:
train_file = "imdb_train.csv"
train_data = pd.read_csv(train_file)

In [3]:
vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(1, 2), sublinear_tf=True, max_features=40000)
features_train = vectorizer.fit_transform(train_data.text)
labels_train = train_data.polarity

In [4]:
model = SGDClassifier(penalty="l1", loss="log", max_iter=200)
model.fit(features_train, labels_train)

c:\python3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)


SGDClassifier(alpha=0.0001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=200,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l1',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False)

In [5]:
model.score(features_train, labels_train)

0.85948

El transformador de features y el modelo se empaquetan en un objeto Pipeline:

In [6]:
pipe = Pipeline([('vectorizer', vectorizer), ('classifier', model)])

In [7]:
test_text = train_data.loc[5, ['text']].tolist()
test_text

['"It appears that many critics find the idea of a Woody Allen drama unpalatable." And for good reason: they are unbearably wooden and pretentious imitations of Bergman. And let\'s not kid ourselves: critics were mostly supportive of Allen\'s Bergman pretensions, Allen\'s whining accusations to the contrary notwithstanding. What I don\'t get is this: why was Allen generally applauded for his originality in imitating Bergman, but the contemporaneous Brian DePalma was excoriated for "ripping off" Hitchcock in his suspense/horror films? In Robin Wood\'s view, it\'s a strange form of cultural snobbery. I would have to agree with that.']

In [8]:
print(pipe.predict_proba(test_text))
print(pipe.predict(test_text))

[[0.69111622 0.30888378]]
[0]


### Serialization

El Pipeline incluye la transformación de los inputs y el modelo en un solo paso:

In [9]:
joblib.dump(pipe, './sentiment_model.pkl')

['./sentiment_model.pkl']

**Uso:**

In [10]:
import numpy as np
from sklearn.externals import joblib

model2 = joblib.load('sentiment_model.pkl')

In [11]:
def scorer(text, classifier):
    pred = classifier.predict(text)[0]
    ppred = np.max(classifier.predict_proba(text))
    return {'sentiment': int(pred), 'probability': float(ppred)}

In [12]:
test_text = train_data.loc[5, ['text']].tolist()
test_text

['"It appears that many critics find the idea of a Woody Allen drama unpalatable." And for good reason: they are unbearably wooden and pretentious imitations of Bergman. And let\'s not kid ourselves: critics were mostly supportive of Allen\'s Bergman pretensions, Allen\'s whining accusations to the contrary notwithstanding. What I don\'t get is this: why was Allen generally applauded for his originality in imitating Bergman, but the contemporaneous Brian DePalma was excoriated for "ripping off" Hitchcock in his suspense/horror films? In Robin Wood\'s view, it\'s a strange form of cultural snobbery. I would have to agree with that.']

In [13]:
result = scorer(text=test_text, classifier=model2)
result

{'sentiment': 0, 'probability': 0.6911162222196799}

<h1><font color="blue">Classification</font></h1>

In [14]:
file = "./iris_data.csv"

In [15]:
iris_data = pd.read_csv(file)
iris_data.head()

,sepal_length,sepal_width,petal_length,petal_width,type
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [16]:
iris_data[iris_data['type'] == 'virginica'].head()

,sepal_length,sepal_width,petal_length,petal_width,type
100,6.3,3.3,6.0,2.5,virginica
101,5.8,2.7,5.1,1.9,virginica
102,7.1,3.0,5.9,2.1,virginica
103,6.3,2.9,5.6,1.8,virginica
104,6.5,3.0,5.8,2.2,virginica


In [17]:
iris_model = LogisticRegression(solver='lbfgs', multi_class='auto')
iris_model.fit(iris_data[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']], iris_data.type)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='auto',
          n_jobs=None, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

In [18]:
iris_model.score(iris_data[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']], iris_data.type)

0.9733333333333334

## Serialization

In [19]:
joblib.dump(iris_model, './iris_model.pkl')

['./iris_model.pkl']

**Uso:**

In [20]:
import numpy as np
from sklearn.externals import joblib

iris_model = joblib.load('./iris_model.pkl')

In [21]:
def scorer(data, model):
    sepal_length, sepal_width = float(data['sepal_length']), float(data['sepal_width'])
    petal_length, petal_width = float(data['petal_length']), float(data['petal_width'])
    # deben ir en el mismo orden en que se estimó el modelo
    data_p = [[sepal_length, sepal_width, petal_length, petal_width]]
    pred = model.predict(data_p)[0]
    ppred = round(np.max(model.predict_proba(data_p)), 4)
    return {'predicted': pred, 'probability': float(ppred), 'message': 'success'}

In [22]:
test_data = {'sepal_length': 4.9, 'sepal_width': 3, 'petal_length': 1.4, 'petal_width': 0.2}
print(scorer(test_data, iris_model))

{'predicted': 'setosa', 'probability': 0.9718, 'message': 'success'}
